In [37]:
import pandas as pd
import numpy as np

In [38]:
def ajuste_colunas_dados_b3(dados_acoes):
    ## Nomear as colunas
    dados_acoes.columns = [
    "tipo_registro",
    "data_pregao",
    "cod_bdi",
    "cod_negociacao",
    "tipo_mercado",
    "noma_empresa",
    "especificacao_papel",
    "prazo_dias_merc_termo",
    "moeda_referencia",
    "preco_abertura",
    "preco_maximo",
    "preco_minimo",
    "preco_medio",
    "preco_ultimo_negocio",
    "preco_melhor_oferta_compra",
    "preco_melhor_oferta_venda",
    "numero_negocios",
    "quantidade_papeis_negociados",
    "volume_total_negociado",
    "preco_exercicio",
    "ìndicador_correcao_precos",
    "data_vencimento" ,
    "fator_cotacao",
    "preco_exercicio_pontos",
    "codigo_isin",
    "num_distribuicao_papel"]

    # Eliminar a última linha
    linha=len(dados_acoes["data_pregao"])
    dados_acoes=dados_acoes.drop(linha-1)

    # Ajustar valores com virgula (dividir os valores dessas colunas por 100)
    listaVirgula=[
    "preco_abertura",
    "preco_maximo",
    "preco_minimo",
    "preco_medio",
    "preco_ultimo_negocio",
    "preco_melhor_oferta_compra",
    "preco_melhor_oferta_venda",
    "volume_total_negociado",
    "preco_exercicio",
    "preco_exercicio_pontos"
    ]

    for coluna in listaVirgula:
        dados_acoes[coluna]=[i/100. for i in dados_acoes[coluna]]
    
    return dados_acoes

In [39]:
def leitura_pontos_de_entrada(ticker,ano):
    filtered_stocks_signals = pd.read_json(r"C:\Users\cptfr\Desktop\TCC_B3_OPCOES\filtered_straddle_results.json")
    resultado = filtered_stocks_signals[(filtered_stocks_signals['Stock'].str.contains(ticker))& (filtered_stocks_signals['Date'].dt.year == ano)]
    return resultado

In [40]:
def filter_entries(group):
    # Resetando o índice do grupo para garantir que a indexação comece de 0. Isso é útil para manipular o DataFrame de forma mais fácil dentro da função.
    group = group.reset_index(drop=True)
    
    # Inicializa uma lista chamada 'keep' com True como primeiro elemento, indicando que a primeira entrada do grupo sempre deve ser mantida.
    keep = [True]  

    # Loop que começa do segundo elemento (índice 1) até o último elemento do grupo.
    for i in range(1, len(group)):
        # Calcula a diferença em dias entre a data da entrada atual e a entrada anterior. Verifica se essa diferença é maior que 20 dias.
        if (group.loc[i, 'Date'] - group.loc[i-1, 'Date']).days > 20:
            # Se a diferença for maior que 20 dias, adiciona True à lista 'keep', indicando que este registro deve ser mantido.
            keep.append(True)
        else:
            # Se a diferença não for maior que 20 dias, adiciona False à lista 'keep',
            # indicando que este registro não deve ser mantido.
            keep.append(False)

    # Retorna apenas os registros do grupo para os quais a lista 'keep' tem o valor True.
    # Isso efetivamente filtra o grupo, mantendo apenas as entradas desejadas.
    return group[keep]

In [41]:
def procura_straddle_alvo(ticker,data,dados_acoes,df_resultados):
    pd.set_option('display.max_rows', None) 
    pd.set_option('display.width', 1000)
    #remover .SA do ticker
    ticker_original = ticker
    ticker = ticker[:-3]
    venc_datas_opcompra = ['A','B','C','D','E','F','G','H','I','J','K','L']
    venc_datas_opvenda = ['M','N','O','P','Q','R','S','T','U','V','W','X']
    #primeiro encontrar o preço a vista na data do ponto de entrada, código 10 para ativos mercado a vista
    filt_avista= dados_acoes[dados_acoes['tipo_mercado']==10][['cod_negociacao','preco_medio','data_pregao']]
    #filtrando por ticker e data da entrada
    filt_avista = filt_avista[(filt_avista['cod_negociacao'].str.contains(ticker, na=False)) & (filt_avista['data_pregao'] == data)]
    #print("Ticker {} e filt_avista {}".format(ticker,filt_avista))
    if ticker in filt_avista['cod_negociacao'].values:
        #print("Ticker a vista existente!")
        #considerando preço a vista como preco médio da data de entrada
        #print(filt_avista)
        preco_avista = filt_avista['preco_medio'].iloc[0]
        print("1)Esse é o preço a vista {} na data {}".format(preco_avista,data))
        #Após encontrar o preço a vista, vamos encontrar o preço da opção mais próxima do strike na data de entrada
        opvenda= dados_acoes[dados_acoes['tipo_mercado']==80][['cod_negociacao','preco_exercicio','preco_medio','data_pregao','data_vencimento']]
        opcompra= dados_acoes[dados_acoes['tipo_mercado']==70][['cod_negociacao','preco_exercicio','preco_medio','data_pregao','data_vencimento']]
        #filtrando as opções por ticker e data do pregão de entrada
        ticker = ticker[:-1] 
        print("2)Ticker Ajustado {}".format(ticker))
        filt_opvenda = opvenda[(opvenda['cod_negociacao'].str.contains(ticker, na=False)) & (opvenda['data_pregao'] == data)]
        filt_opcompra = opcompra[(opcompra['cod_negociacao'].str.contains(ticker, na=False)) & (opcompra['data_pregao'] == data)]
        if not filt_opcompra.empty:
            #menor data de vencimento
            min_date = filt_opcompra['data_vencimento'].min()
            filt_opcompra_data_vencimento = filt_opcompra[filt_opcompra['data_vencimento'] == min_date]
            #Encontrando o strike com menor diferença absoluta para o ativo a vista
            diferenca = np.abs(filt_opcompra_data_vencimento['preco_exercicio'] - preco_avista)
            #print("Diferença {}".format(diferenca))
            #Encontrar o índice do elemento que tem a menor diferença absoluta
            idx = diferenca.idxmin()
            strike_mais_proximo = filt_opcompra['preco_exercicio'].loc[idx]
            print("3)Strike mais proximo {} {}".format(idx,strike_mais_proximo))
            datavencimento = int(filt_opcompra['data_vencimento'].loc[idx])
            #ticker da opção mais próxima do preço ativo
            ticker_op = filt_opcompra['cod_negociacao'].loc[idx]
            print("4)Ticker opção mais proxima {} com vencimento {}".format(ticker_op,datavencimento))
            #pegando o quarto elemento do ticker, que é a letra que faz referência ao mês de vencimento
            ticker_venc_op = ticker_op[4]
            #print("Data de entrada {} Data do vencimento {} e ticker da opcao {} Letra mês de vencimento {}".format(data,datavencimento, ticker_op, ticker_venc_op))
            #percorre a lista venc_datas_opcompra e encontra a letra igual a do mês de vencimento
            for letra in range(len(venc_datas_opcompra)):
            #print(letra)
                if venc_datas_opcompra[letra] == ticker_venc_op:
                    #verifica se a posição da letra encontrada é a última da lista, se for volta para a posição 0
                    if venc_datas_opcompra[-1] == ticker_venc_op:
                        posicao_prox_venc = 0
                    #caso contrário só soma uma posição para pegar a opção de próximo vencimento
                    else: 
                        posicao_prox_venc = letra + 1
            #Agora será calculado o strike mais proximo do ativo a vista, mas para a opção com vencimento no mês seguinte
            #uniao do ticker com a letra do vencimento
            #print(venc_datas_opcompra[posicao_prox_venc])
            #print(venc_datas_opvenda[posicao_prox_venc])
            ticker_op_venda = ticker + venc_datas_opvenda[posicao_prox_venc]
            ticker_op_compra = ticker + venc_datas_opcompra[posicao_prox_venc]
            print("5) Ticker compra novo vencimento {} e Ticker venda novo vencimento {}".format(ticker_op_compra,ticker_op_venda))
            existe_ticker_compra = any(ticker.startswith(ticker_op_compra) for ticker in filt_opcompra['cod_negociacao'].values)
            existe_ticker_venda = any(ticker.startswith(ticker_op_venda) for ticker in filt_opvenda['cod_negociacao'].values)
            if existe_ticker_venda and existe_ticker_compra:
                print("Existe opções com próximo vencimento disponível!")
                #Filtra as opções pelo ticker da opção alvo na data de entrada
                filt_op_alvo_compra = opcompra[(opcompra['cod_negociacao'].str.contains(ticker_op_compra, na=False)) & (opcompra['data_pregao'] == data)]
                filt_op_alvo_venda =  opvenda[(opvenda['cod_negociacao'].str.contains(ticker_op_venda, na=False)) & (opvenda['data_pregao'] == data)]
                result = pd.merge(filt_op_alvo_compra,filt_op_alvo_venda, on=['preco_exercicio','data_vencimento'])
                if not result.empty:
                    print(result)
                    diferenca = np.abs(result['preco_exercicio'] - preco_avista)
                    #Encontrar o índice do elemento que tem a menor diferença absoluta
                    idx = diferenca.idxmin()
                    #Obtenção do strike mais proximo 
                    strike_mais_proximo_op_alvo = result['preco_exercicio'].loc[idx]
                    ticker_op_alvo_compra = result['cod_negociacao_x'].loc[idx]
                    ticker_op_alvo_venda = result['cod_negociacao_y'].loc[idx]
                    datavencimento = int(result['data_vencimento'].loc[idx])
                    preco_opcao_compra = result['preco_medio_x'].loc[idx]
                    preco_opcao_venda = result['preco_medio_y'].loc[idx]
                    filt_opcompra_alvo = opcompra[(opcompra['cod_negociacao'].str.contains(ticker_op_alvo_compra, na=False)) & (opcompra['data_pregao'] >= data) & (opcompra['data_pregao'] <= str(datavencimento))]
                    filt_opvenda_alvo = opvenda[(opvenda['cod_negociacao'].str.contains(ticker_op_alvo_venda, na=False)) & (opvenda['data_pregao'] >= data) & (opvenda['data_pregao'] <= str(datavencimento))]
                    preco_opcao_compra_vencimento = filt_opcompra_alvo['preco_medio'].iloc[-1]
                    preco_opcao_venda_vencimento = filt_opvenda_alvo['preco_medio'].iloc[-1]
                    resultado = (preco_opcao_compra_vencimento - preco_opcao_compra) + (preco_opcao_venda_vencimento - preco_opcao_venda)
                    retorno = round((resultado / (preco_opcao_compra + preco_opcao_venda)) *100,2)
                    print("RETORNO {}".format(retorno))
                    if retorno >= 30:
                        print("Retorno deu 30%")
                        retorno = 30.00
                    elif retorno < -30:
                        print("Retorno deu -30% ")
                        retorno = -30.00
                    else:
                        print("Retorno deu {}".format(retorno))
                    new_index = len(df_resultados)
                    df_resultados.loc[new_index,'Data_Entrada'] = data
                    df_resultados.loc[new_index,'Data_Vencimento'] = datavencimento
                    df_resultados.loc[new_index,'Tipo'] = 'STRADDLE'
                    df_resultados.loc[new_index,'Ativo'] = ticker_original
                    df_resultados.loc[new_index, 'ResultadosTotais'] = retorno
                else: 
                    print("Sem strikes em comum")
            else:
                print("Opção com vencimento mais longo indisponível")
        else: 
            print("Sem opções de venda disponíveis para o período")
    else: 
        print("Ticker Não Encontrado")
    return df_resultados
    

In [42]:
#anos = ['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011',
#        '2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']

anos = ['2016','2017','2018','2019','2020','2021','2022','2023']
df_resultados = pd.DataFrame(columns=("Data_Entrada","Data_Vencimento","Tipo",'Ativo','ResultadosTotais'))
tickers = ["PETR4.SA", 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC3.SA']

for ano in anos:
        print(ano)
        ## Séries históricas disponíveis em
        ## http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/
        arquivo_bovespa = r'C:\Users\cptfr\Desktop\TCC_B3_OPCOES\COTAHIST_A{}.TXT'.format(ano)
        ## Estrutura do arquivo disponível em 
        ## http://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf

        tamanho_campos=[2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]
        dados_acoes=pd.read_fwf(arquivo_bovespa, widths=tamanho_campos, header=0, encoding = 'latin1')
        dados_acoes = ajuste_colunas_dados_b3(dados_acoes)
        for ticker in tickers:
                resultado = leitura_pontos_de_entrada(ticker,int(ano))
                resultado_filtrado = resultado.groupby([resultado['Stock'], resultado['Date'].dt.to_period('M')],group_keys=True).apply(filter_entries).reset_index(drop=True)
                for linha in range(len(resultado_filtrado)):
                        data = str(resultado_filtrado['Date'].loc[linha].date())
                        data = data.replace("-","")
                        df_resultados = procura_straddle_alvo(ticker,data,dados_acoes,df_resultados)
                        print(df_resultados)
                        
                #print(resultado_filtrado)        
        #break
arquivo_json = 'resultados_straddle.json'

try:
    dados_existentes = pd.read_json(arquivo_json)
except ValueError:  # Trata o erro se o JSON estiver vazio ou corrompido
    dados_existentes = pd.DataFrame()

# Concatenar os dados existentes com os novos dados
dados_atualizados = pd.concat([dados_existentes, df_resultados], ignore_index=True)

# Salvar o DataFrame atualizado de volta ao arquivo JSON
dados_atualizados.to_json(arquivo_json, orient='records')
print(df_resultados)

    #print(dados_acoes['cod_negociacao'])

2016
1)Esse é o preço a vista 13.28 na data 20160919
2)Ticker Ajustado PETR
3)Strike mais proximo 336615 13.25
4)Ticker opção mais proxima PETRJ36 com vencimento 20161017
5) Ticker compra novo vencimento PETRK e Ticker venda novo vencimento PETRW
Existe opções com próximo vencimento disponível!
  cod_negociacao_x  preco_exercicio  preco_medio_x data_pregao_x  data_vencimento cod_negociacao_y  preco_medio_y data_pregao_y
0          PETRK11             11.0           2.83      20160919       20161121.0          PETRW11           0.20      20160919
1          PETRK13             13.0           1.31      20160919       20161121.0          PETRW13           0.76      20160919
2          PETRK14             14.0           0.90      20160919       20161121.0          PETRW14           1.25      20160919
3           PETRK2             12.0           1.92      20160919       20161121.0           PETRW2           0.44      20160919
4          PETRK37              9.7           3.71      20160919